## Variant data selection and preprocessing (__ClinVar__)

### Load and filter ClinVar missense variants

In [1]:
# FTP site:              https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/
# Donwloaded file:       variant_summary.txt.gz	last modified: 2025-02-09

import VEP
import gzip
import io
import numpy as np
import pandas as pd
import re
import warnings
from Bio.Data import IUPACData
from collections import Counter

warnings.filterwarnings("ignore")

### Load data

In [2]:
data = pd.read_csv('../data/clinvar/variant_summary.txt', sep='\t')

In [3]:
data.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),...,AlternateAlleleVCF,SomaticClinicalImpact,SomaticClinicalImpactLastEvaluated,ReviewStatusClinicalImpact,Oncogenicity,OncogenicityLastEvaluated,ReviewStatusOncogenicity,SCVsForAggregateGermlineClassification,SCVsForAggregateSomaticClinicalImpact,SCVsForAggregateOncogenicityClassification
0,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 25, 2024",397704705,...,TGCTGTAAACTGTAACTGTAAA,-,-,-,-,-,-,SCV001451119|SCV005622007,-,-
1,15041,Indel,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...,9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 25, 2024",397704705,...,TGCTGTAAACTGTAACTGTAAA,-,-,-,-,-,-,SCV001451119|SCV005622007,-,-
2,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,G,-,-,-,-,-,-,SCV000020156,-,-
3,15042,Deletion,NM_014855.3(AP5Z1):c.1413_1426del (p.Leu473fs),9907,AP5Z1,HGNC:22197,Pathogenic,1,"Jun 29, 2010",397704709,...,G,-,-,-,-,-,-,SCV000020156,-,-
4,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,0,"Jun 29, 2015",150829393,...,A,-,-,-,-,-,-,SCV000020157,-,-


In [4]:
len(data)

6548628

In [5]:
for count, col in enumerate(data.columns, start=1):
    print(count, col)

1 #AlleleID
2 Type
3 Name
4 GeneID
5 GeneSymbol
6 HGNC_ID
7 ClinicalSignificance
8 ClinSigSimple
9 LastEvaluated
10 RS# (dbSNP)
11 nsv/esv (dbVar)
12 RCVaccession
13 PhenotypeIDS
14 PhenotypeList
15 Origin
16 OriginSimple
17 Assembly
18 ChromosomeAccession
19 Chromosome
20 Start
21 Stop
22 ReferenceAllele
23 AlternateAllele
24 Cytogenetic
25 ReviewStatus
26 NumberSubmitters
27 Guidelines
28 TestedInGTR
29 OtherIDs
30 SubmitterCategories
31 VariationID
32 PositionVCF
33 ReferenceAlleleVCF
34 AlternateAlleleVCF
35 SomaticClinicalImpact
36 SomaticClinicalImpactLastEvaluated
37 ReviewStatusClinicalImpact
38 Oncogenicity
39 OncogenicityLastEvaluated
40 ReviewStatusOncogenicity
41 SCVsForAggregateGermlineClassification
42 SCVsForAggregateSomaticClinicalImpact
43 SCVsForAggregateOncogenicityClassification


We can highlight diverse columns that will be important for the next steps:
- __AlleleID (1)__: Unique identifier for each allele in ClinVar.
- __GeneSymbol (5)__: Identifies the affected gene. 
- __ClinicalSignificance (7)__: Defines pathogenicity classification (e.g., benign, pathogenic, etc).
- __ClinSigSimple (8)__: Simplified numeric version of previous column (e.g., 0 and 1).
- __LastEvaluated (9)__: Date of the last variant classification update.
- __ReviewStatus (25)__: Indicates the confidence level of the variant classification.
- __Chromosome (19)__ and __PositionVCF (32)__: Gives variant location.
- __ReferenceAlleleVCF (33)__ and __AlternateAlleleVCF (34)__: Specifies the genetic variant.

In [6]:
data[["#AlleleID", "GeneSymbol", "ClinicalSignificance", "ClinSigSimple", 
      "LastEvaluated", "ReviewStatus", "Chromosome", "PositionVCF", 
      "ReferenceAlleleVCF", "AlternateAlleleVCF"]].head()

,#AlleleID,GeneSymbol,ClinicalSignificance,ClinSigSimple,LastEvaluated,ReviewStatus,Chromosome,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15041,AP5Z1,Pathogenic,1,"Jun 25, 2024","criteria provided, multiple submitters, no con...",7,4820844,GGAT,TGCTGTAAACTGTAACTGTAAA
1,15041,AP5Z1,Pathogenic,1,"Jun 25, 2024","criteria provided, multiple submitters, no con...",7,4781213,GGAT,TGCTGTAAACTGTAACTGTAAA
2,15042,AP5Z1,Pathogenic,1,"Jun 29, 2010",no assertion criteria provided,7,4827360,GCTGCTGGACCTGCC,G
3,15042,AP5Z1,Pathogenic,1,"Jun 29, 2010",no assertion criteria provided,7,4787729,GCTGCTGGACCTGCC,G
4,15043,ZNF592,Uncertain significance,0,"Jun 29, 2015",no assertion criteria provided,15,85342440,G,A


### Filter by criteria

### 1. Keep only SNVs (remove insertions, deletions, complex variants).

In [7]:
data['Type'].unique()

array(['Indel', 'Deletion', 'single nucleotide variant', 'Duplication',
       'Microsatellite', 'Insertion', 'Variation', 'Complex',
       'Translocation', 'Inversion', 'copy number gain', 'fusion',
       'copy number loss', 'protein only', 'Tandem duplication'],
      dtype=object)

In [8]:
data_filter1 = data[data.Type =='single nucleotide variant'].reset_index(drop=True)
len(data_filter1)

5944878

##### --> Parse "Name" column and filter variants

In [9]:
# Convert the 3-letter Aa codes to 1-letter codes
three_to_one = IUPACData.protein_letters_3to1

print(three_to_one.keys())
print(three_to_one.values())

dict_keys(['Ala', 'Cys', 'Asp', 'Glu', 'Phe', 'Gly', 'His', 'Ile', 'Lys', 'Leu', 'Met', 'Asn', 'Pro', 'Gln', 'Arg', 'Ser', 'Thr', 'Val', 'Trp', 'Tyr'])
dict_values(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])


In [10]:
# Function to extract the variant in 3-letter and 1-letter format
def extract_variant(name):
    # find protein variant descriptions in the Name column
    match = re.search(r'p\.([A-Za-z]+[0-9]+[A-Za-z]+)', name)    # e.g., p.Arg123His
    if match:
        # extract full variant string (e.g., "Arg123His")
        three_letter_variant = match.group(1)
        # convert the 3-letter variant to 1-letter variant
        variant_3letter = three_letter_variant

        # first 3 letters (original Aa)
        three_letter_from = three_letter_variant[:3]
        # extract the numeric position
        position = re.search(r'\d+', three_letter_variant).group()
        # last 3 letters (mutated Aa)
        three_letter_to = three_letter_variant[-3:]

        # convert 3-letter to 1-letter (e.g., "Arg123His" to "R123H")
        one_letter_from = three_to_one.get(three_letter_from.capitalize(), three_letter_from)
        one_letter_to = three_to_one.get(three_letter_to.capitalize(), three_letter_to)
        variant = f"{one_letter_from}{position}{one_letter_to}" 

        return variant_3letter, variant   # e.g., "Arg123His", "R123H"

    else:
        return None, None

In [11]:
# Addition of 2 new cols
data_filter1[['Variant (3-letter)', 'Variant']] = data_filter1['Name'].apply(
    lambda x: pd.Series(extract_variant(x)))

In [12]:
data_filter1[["#AlleleID", "GeneSymbol", "Variant (3-letter)", "Variant"]].head()

,#AlleleID,GeneSymbol,Variant (3-letter),Variant
0,15043,ZNF592,Gly1046Arg,G1046R
1,15043,ZNF592,Gly1046Arg,G1046R
2,15044,FOXRED1,Gln232Ter,Q232Ter
3,15044,FOXRED1,Gln232Ter,Q232Ter
4,15045,FOXRED1,Asn430Ser,N430S


In [13]:
# Function to check if both parts of the variant are valid Aas
def is_valid_variant(variant_3letter):
    valid_amino_acids = set(three_to_one.keys())  # set of valid 3-letter Aas

    if pd.isna(variant_3letter):
        return False

    # extract the 3-letter Aa codes (first 3 and last 3 chars)
    three_letter_from = variant_3letter[:3]
    three_letter_to = variant_3letter[-3:]
    
    # check if both are in the valid set and no "Ter" (stop codon)
    if (three_letter_from in valid_amino_acids and three_letter_to in valid_amino_acids 
        and 'Ter' not in variant_3letter):
        return True
    else:
        return False

In [14]:
data_filter1_1 = data_filter1[data_filter1['Variant (3-letter)'].apply(is_valid_variant)]

In [15]:
data_filter1_1[["#AlleleID", "GeneSymbol", "Variant (3-letter)", "Variant"]].head()

,#AlleleID,GeneSymbol,Variant (3-letter),Variant
0,15043,ZNF592,Gly1046Arg,G1046R
1,15043,ZNF592,Gly1046Arg,G1046R
4,15045,FOXRED1,Asn430Ser,N430S
5,15045,FOXRED1,Asn430Ser,N430S
6,15046,NUBPL,Gly56Arg,G56R


In [16]:
# invalid or ambiguous variants were removed
len(data_filter1_1)

3426329

### 2. Retain submissions from 2021 and later.

In [17]:
data_filter1_1[['LastEvaluated']].head()

,LastEvaluated
0,"Jun 29, 2015"
1,"Jun 29, 2015"
4,"Jun 06, 2024"
5,"Jun 06, 2024"
6,"Jul 05, 2022"


In [18]:
# convert LastEvaluated column to datetime format
data_filter1_1['LastEvaluated'] = pd.to_datetime(data_filter1_1['LastEvaluated'], errors='coerce')

# new column extracting the year and converting it to integer
data_filter1_1['LastEvaluated (Year)'] = data_filter1_1['LastEvaluated'].dt.year.astype('Int64')

In [19]:
data_filter1_1[['LastEvaluated', 'LastEvaluated (Year)']].head()

,LastEvaluated,LastEvaluated (Year)
0,2015-06-29,2015
1,2015-06-29,2015
4,2024-06-06,2024
5,2024-06-06,2024
6,2022-07-05,2022


In [20]:
# filter variants where year is 2021 or later
data_filter2 = data_filter1_1[data_filter1_1['LastEvaluated (Year)'] >= 2021]

In [21]:
data_filter2[['LastEvaluated', 'LastEvaluated (Year)']].head()

,LastEvaluated,LastEvaluated (Year)
4,2024-06-06,2024
5,2024-06-06,2024
6,2022-07-05,2022
7,2022-07-05,2022
8,2024-11-01,2024


In [22]:
len(data_filter2)

3112595

In [23]:
data_filter1_1.groupby('LastEvaluated (Year)').size()

LastEvaluated (Year)
1965          2
1973          2
1976          2
1977          2
1979          2
1980          4
1981          4
1982          2
1983          8
1984         10
1985          4
1986         10
1987         14
1988         16
1989         60
1990         80
1991        114
1992        192
1993        176
1994        176
1995        240
1996        166
1997        226
1998        282
1999        294
2000        314
2001        494
2002        393
2003        374
2004        384
2005        288
2006        392
2007        442
2008        424
2009        494
2010        540
2011        984
2012       1620
2013       4998
2014       4900
2015       7295
2016      16063
2017      28332
2018      61288
2019      76755
2020      71901
2021     344526
2022     766916
2023     963595
2024    1034214
2025       3344
dtype: int64

In [24]:
data_filter2.groupby('LastEvaluated (Year)').size()

LastEvaluated (Year)
2021     344526
2022     766916
2023     963595
2024    1034214
2025       3344
dtype: int64

### 3. Exclude variants with zero-star review status, VUS and conflicting classifications.

##### --> Remove VUS and conflicting classifications

In [25]:
data_filter2.ClinicalSignificance.unique()

array(['Likely pathogenic',
       'Conflicting classifications of pathogenicity',
       'Pathogenic/Pathogenic, low penetrance; other; risk factor',
       'Pathogenic/Likely pathogenic/Pathogenic, low penetrance; other',
       'Uncertain significance', 'Pathogenic/Likely pathogenic',
       'Pathogenic', 'Likely benign', 'Benign', 'Benign/Likely benign',
       'Conflicting classifications of pathogenicity; risk factor',
       'drug response', 'Benign; drug response',
       'Conflicting classifications of pathogenicity; association; risk factor',
       'Benign/Likely benign; other',
       'Conflicting classifications of pathogenicity; other',
       'Pathogenic/Likely pathogenic; risk factor',
       'no classifications from unflagged records', 'not provided',
       'Likely benign; other', 'Benign; other', 'Pathogenic; risk factor',
       'Conflicting classifications of pathogenicity; association',
       'Benign/Likely benign; association', 'Benign; risk factor',
       'Ben

In [26]:
data_filter3 = data_filter2[data_filter2['ClinicalSignificance'].isin(['Pathogenic','Likely pathogenic',
                                                                       'Pathogenic/Likely pathogenic', 
                                                                       'Benign', 'Likely benign', 
                                                                       'Benign/Likely benign'])]

In [27]:
data_filter3.ClinicalSignificance.unique()

array(['Likely pathogenic', 'Pathogenic/Likely pathogenic', 'Pathogenic',
       'Likely benign', 'Benign', 'Benign/Likely benign'], dtype=object)

In [28]:
len(data_filter3)

277997

##### --> Filter out variants with zero-star review status

In [29]:
# somatic classification
data_filter3.ReviewStatusClinicalImpact.unique()

array(['-', 'criteria provided, multiple submitters',
       'no assertion criteria provided',
       'criteria provided, single submitter'], dtype=object)

In [30]:
# germline classification
data_filter3.ReviewStatus.unique()

array(['criteria provided, single submitter',
       'criteria provided, multiple submitters, no conflicts',
       'reviewed by expert panel', 'no assertion criteria provided'],
      dtype=object)

In [31]:
data_filter3.ReviewStatus.value_counts()

ReviewStatus
criteria provided, single submitter                     176317
criteria provided, multiple submitters, no conflicts     86823
no assertion criteria provided                           10115
reviewed by expert panel                                  4742
Name: count, dtype: int64

In [32]:
data_filter3_1 = data_filter3[data_filter3['ReviewStatus'] != 'no assertion criteria provided'].reset_index(drop=True)

In [33]:
data_filter3_1.ReviewStatus.value_counts()

ReviewStatus
criteria provided, single submitter                     176317
criteria provided, multiple submitters, no conflicts     86823
reviewed by expert panel                                  4742
Name: count, dtype: int64

In [34]:
len(data_filter3_1)

267882

##### --> Parse Chromosome column

In [35]:
data_filter3_1.Chromosome.unique()

array(['11', '6', '10', '16', '22', '15', '7', '1', '8', '21', '5', '19',
       '4', '3', '17', '12', '20', '9', '18', '2', '14', '13', 'MT', 'Y',
       'X', 'na', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 'Un'], dtype=object)

In [36]:
data_filter3_2=data_filter3_1[~data_filter3_1.Chromosome.isin(['na','Un'])].reset_index(drop=True)

In [37]:
# after removing invalid or missing chromosome data
len(data_filter3_2)

267735

In [38]:
data_filter3_2.Chromosome.unique()

array(['11', '6', '10', '16', '22', '15', '7', '1', '8', '21', '5', '19',
       '4', '3', '17', '12', '20', '9', '18', '2', '14', '13', 'MT', 'Y',
       'X', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22], dtype=object)

In [39]:
data_filter3_2['Chromosome'] = data_filter3_2['Chromosome'].astype(str)
data_filter3_2['Chromosome'].nunique()

25

In [40]:
data_filter3_2['Chromosome'].unique()

array(['11', '6', '10', '16', '22', '15', '7', '1', '8', '21', '5', '19',
       '4', '3', '17', '12', '20', '9', '18', '2', '14', '13', 'MT', 'Y',
       'X'], dtype=object)

Now the 'Chromosome' column contains the expected chromosome identifiers:
- autosomes (1-22)
- sex chromosomes (X, Y)
- mitochondrial DNA (MT)

### 4. Keep variants with AF<0.01 in the gnomAD v.2.1.

At this point, we need to pass the data through VEP in order to get gnomAD information.

But first, the __assembly issue__ must be taken into account.

In [97]:
data_filter3_2.Assembly.unique()

array(['GRCh38', 'GRCh37'], dtype=object)

In [42]:
data_filter3_2 = data_filter3_2.sort_values(by='Assembly', ascending=False).reset_index(drop=True)
data_filter3_2.Assembly.value_counts()

Assembly
GRCh37    133870
GRCh38    133865
Name: count, dtype: int64

Here, we notice that several variants have the same Allele ID and the only difference is the Assembly. 

Thus, we make sure this is the case and then, drop duplicates.

In [43]:
# Function to identify Allele ID groups with inconsistent data
def check_duplicates(df):
    cols_to_check=['#AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol', 'HGNC_ID',
       'ClinicalSignificance', 'ClinSigSimple', 'LastEvaluated', 'RS# (dbSNP)',
       'nsv/esv (dbVar)', 'RCVaccession', 'PhenotypeIDS', 'PhenotypeList',
       'Origin', 'OriginSimple', 'Cytogenetic', 'ReviewStatus', 
       'NumberSubmitters', 'Guidelines', 'TestedInGTR', 'OtherIDs', 
       'SubmitterCategories', 'VariationID', 'Variant (3-letter)', 'Variant']

    grouped = df.groupby('#AlleleID')
    
    # find groups where any column (except Assembly) has different values
    different_values = {}
    for name, group in grouped:
        if len(group) > 1:  # only check groups with multiple variants
            for col in cols_to_check:
                unique_values = group[col].nunique()
                if unique_values > 1:
                    if name not in different_values:
                        different_values[name] = []
                    different_values[name].append(col)

    if different_values:
        print("Found AlleleIDs with different values in columns other than Assembly:")
        for allele_id, columns in different_values.items():
            print(f"\nAlleleID {allele_id} has different values in columns: {columns}")
    else:
        print("All rows with the same #AlleleID have identical values (except possibly Assembly)")

    return df['#AlleleID'].duplicated().sum()

In [44]:
total_duplicates= check_duplicates(data_filter3_2)

All rows with the same #AlleleID have identical values (except possibly Assembly)


In [98]:
data_filter4 = data_filter3_2.drop_duplicates(subset=['#AlleleID'],keep='first')
data_filter4.Assembly.value_counts()

Assembly
GRCh38    133845
GRCh37        10
Name: count, dtype: int64

Next, we must separate data by Assembly before using VEP. 

This is done because coordinates info should __not__ be mixed.

In [46]:
for assembly in ['GRCh37', 'GRCh38']:
    file = data_filter4[data_filter4['Assembly'] == assembly].copy()
    file.to_csv(f'../data/clinvar/clinvar_data_preVEP_{assembly.lower()}.csv', index=0)

VEP input must be in VCF format, thus the following function.

In [47]:
def create_vcf(df, outputfile):
    # necessary columns for VCF format
    vcf_columns = [
        'Chromosome',
        'PositionVCF',
        'RS# (dbSNP)',
        'ReferenceAlleleVCF',
        'AlternateAlleleVCF'
    ]

    vcf_df = df[vcf_columns].copy()
    vcf_df['Chromosome'] = vcf_df['Chromosome'].astype(str)
    vcf_df['Chromosome'] = vcf_df['Chromosome'].str.replace('chr', '', case=False)

    chrom_order = ([str(i) for i in range(1, 23)] + ['X', 'Y', 'MT'])

    vcf_df['Chromosome'] = pd.Categorical(vcf_df['Chromosome'], categories=chrom_order, ordered=True)
    vcf_df['PositionVCF'] = pd.to_numeric(vcf_df['PositionVCF'])
    vcf_df = vcf_df.sort_values(['Chromosome', 'PositionVCF'])
    
    # add remaining required VCF columns
    vcf_df['ID'] = vcf_df['RS# (dbSNP)'].fillna('.')
    vcf_df['QUAL'] = '.'  
    vcf_df['FILTER'] = '.'  
    vcf_df['INFO'] = '.'  
    
    # reorder and rename columns to match VCF format
    vcf_df = vcf_df[['Chromosome', 'PositionVCF', 'ID', 'ReferenceAlleleVCF', 'AlternateAlleleVCF', 'QUAL', 'FILTER', 'INFO']]
    vcf_df.columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']

    vcf_df.to_csv(outputfile, sep='\t', index=False)
    print(f"VCF file created at: {outputfile}")

After running VEP for the third filter criteria (gnomAD), we should rearrange the output and continue cleaning the dataset.

In [48]:
def parse_output(file_path):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if line.startswith("#Uploaded_variation"):
                header_line = i
                break

    df = pd.read_csv(file_path, delimiter='\t', skiprows=header_line,low_memory=False)
    #print(df.columns)
    print('initial file length:', len(df))

    # rename the first column (Uploaded_variation)
    df.columns = df.columns.str.replace('#', '')
    
    df2= df[df.Protein_position != '-'].reset_index(drop=True).copy()

    df_filtered = df2[df2['Amino_acids'].str.contains('/')]
    df_filtered = df_filtered[~df_filtered['Amino_acids'].str.contains(r'\*')]

    # create Chromosome, PositionVCF, variant columns before merging with the original clinvar df
    df_filtered[['Chromosome', 'PositionVCF_dashed']] = df_filtered['Location'].str.split(':', expand=True)
    df_filtered['PositionVCF'] = df_filtered.apply(lambda x: x['PositionVCF_dashed'].split('-')[0], axis=1)
    
    df_filtered['PositionVCF'] = df_filtered['PositionVCF'].astype(int)
    df_filtered['Variant'] = df_filtered.apply(lambda x: x['Amino_acids'].split('/')[0] + str(x['Protein_position']) + x['Amino_acids'].split('/')[1], axis=1)
    
    # the Allele column represents the alternate allele for a variant. 
    # the Reference allele is not explicitly listed here because VCF files generally only list 
        # the alternate allele (the variation from the reference genome). 
    # the Reference allele would typically be implied based on the genomic position and reference sequence.
    
    df_filtered['AlternateAlleleVCF'] = df_filtered['Allele']

    df_filtered2 = df_filtered[df_filtered['Consequence'].str.contains('missense_variant', na=False)].copy()
    
    # duplicated lines, due to canonical + isoform. we drop duplicates based on 
    df_filtered3 = df_filtered2.drop_duplicates(subset=['Chromosome','PositionVCF','Allele','Gene','Feature_type','CDS_position','Protein_position','Variant'], keep='first')
    df_filtered3['GeneSymbol'] = df_filtered3['Extra'].str.extract(r'SYMBOL=([^;]+)')
    df_filtered3['HGNC_ID'] = df_filtered3['Extra'].str.extract(r'HGNC_ID=([^;]+)')
    
    print('final file length:',len(df_filtered3))
    
    return df_filtered3


def merge_original_and_vepout(df1,df2):
    df1['Chromosome'] = df1['Chromosome'].astype(str)
    df2['Chromosome'] = df2['Chromosome'].astype(str)
    df1=df1.reset_index(drop=True)
    df2=df2.reset_index(drop=True)
    merged_df = df1.merge(df2, on =['Chromosome','PositionVCF','AlternateAlleleVCF','Variant','GeneSymbol','HGNC_ID'], how='left')
    return merged_df    

First, for GRCh38:

In [49]:
data_grch38 = pd.read_csv('../data/clinvar/clinvar_data_preVEP_grch38.csv')
create_vcf(data_grch38, '../data/clinvar/clinvar_data_inputVEP_grch38.vcf')

VCF file created at: ../data/clinvar/clinvar_data_inputVEP_grch38.vcf


To run VEP, the following command was executed:


In [50]:
#  ./vep -i clinvar_data_inputVEP_grch38.vcf -o clinvar_data_outputVEP_grch38.txt --offline \
#      --assembly GRCh38 \
#      --symbol --transcript_version --ccds --protein --uniprot \
#      --hgvs --fasta Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz \
#      --af --af_1kg --af_gnomade --af_gnomadg --max_af

In [51]:
output_grch38 = "../data/clinvar/clinvar_data_outputVEP_grch38.txt"
df = parse_output(output_grch38)

data_filter4_1_grch38 = merge_original_and_vepout(data_grch38, df)

# same rows, but more columns after merging
len(data_filter4_1_grch38) == len(data_grch38)

initial file length: 1613017
final file length: 339455


True

In [52]:
data_filter4_1_grch38.Consequence.unique()

array(['missense_variant', nan, 'missense_variant,splice_region_variant',
       'missense_variant,NMD_transcript_variant',
       'missense_variant,splice_region_variant,NMD_transcript_variant'],
      dtype=object)

In [53]:
data_filter4_1_grch38['gnomADe_AF'] = data_filter4_1_grch38['Extra'].str.extract(r'gnomADe_AF=([^;]+)')
data_filter4_1_grch38['gnomADg_AF'] = data_filter4_1_grch38['Extra'].str.extract(r'gnomADg_AF=([^;]+)')

data_filter4_1_grch38[['gnomADe_AF','gnomADg_AF']]

,gnomADe_AF,gnomADg_AF
0,0.001133,0.0009133
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
133840,0.02981,0.01941
133841,0.0003079,0.002903
133842,0.0003872,0.0001838
133843,NaN,NaN


In [54]:
len(data_filter4_1_grch38[data_filter4_1_grch38.gnomADe_AF.notna()])

101504

Then, for GRCh37:

In [55]:
data_grch37 = pd.read_csv('../data/clinvar/clinvar_data_preVEP_grch37.csv')
create_vcf(data_grch37, '../data/clinvar/clinvar_data_inputVEP_grch37.vcf')

VCF file created at: ../data/clinvar/clinvar_data_inputVEP_grch37.vcf


To run VEP, the following command was executed:


In [56]:
#  ./vep -i clinvar_data_inputVEP_grch37.vcf -o clinvar_data_outputVEP_grch37.txt --offline \
#      --assembly GRCh37 \
#      --symbol --transcript_version --ccds --protein --uniprot \
#      --hgvs --fasta Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz \
#      --af --af_1kg --af_gnomade --af_gnomadg --max_af

In [57]:
output_grch37 = "../data/clinvar/clinvar_data_outputVEP_grch37.txt"
df = parse_output(output_grch37)
df['HGNC_ID'] = df['HGNC_ID'].apply(lambda x: 'HGNC:' + str(x) if pd.notna(x) else x)

data_filter4_1_grch37 = merge_original_and_vepout(data_grch37, df)

# same rows, but more columns after merging
len(data_filter4_1_grch37) == len(data_grch37)

initial file length: 20
final file length: 12


True

In [58]:
data_filter4_1_grch37.Consequence.unique()

array(['missense_variant'], dtype=object)

In [59]:
data_filter4_1_grch37['gnomADe_AF'] = data_filter4_1_grch37['Extra'].str.extract(r'gnomADe_AF=([^;]+)')

len(data_filter4_1_grch37[data_filter4_1_grch37.gnomADe_AF.notna()])

10

Finally, we can merge again GRCh37 and GRCh38 files

In [60]:
# cols to mantain, from original dataset
cols = ['#AlleleID', 'Type', 'Name', 'GeneID', 'GeneSymbol', 'HGNC_ID', 
        'ClinicalSignificance', 'ClinSigSimple', 'RS# (dbSNP)', 'nsv/esv (dbVar)', 
        'RCVaccession', 'PhenotypeIDS', 'PhenotypeList', 'Origin', 'OriginSimple', 
        'Assembly', 'ChromosomeAccession', 'Chromosome', 'Start', 'Stop', 'Cytogenetic', 
        'ReviewStatus', 'NumberSubmitters', 'OtherIDs', 'SubmitterCategories', 
        'VariationID', 'PositionVCF', 'ReferenceAlleleVCF', 'AlternateAlleleVCF', 
        'Variant (3-letter)', 'Variant', 'LastEvaluated (Year)']

# cols to add, after obtaining gnomAD info
cols_to_take = ['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 
                'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 
                'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 
                'Extra', 'PositionVCF_dashed', 'gnomADe_AF']

for col in cols_to_take:
    cols.append(col)

len(cols)

48

In [61]:
data_filter4_2 = pd.concat([data_filter4_1_grch38,data_filter4_1_grch37])
data_filter4_2 = data_filter4_2[cols]
len(data_filter4_2)
data_filter4_2.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,RS# (dbSNP),nsv/esv (dbVar),...,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,Extra,PositionVCF_dashed,gnomADe_AF
0,1809239,single nucleotide variant,NM_001386125.1(OBSCN):c.496G>A (p.Ala166Thr),84033,OBSCN,HGNC:15719,Benign/Likely benign,0,-1,-,...,missense_variant,637,496,166,A/T,Gca/Aca,rs555146765,IMPACT=MODERATE;STRAND=1;SYMBOL=OBSCN;SYMBOL_S...,228212279,0.001133
1,2059503,single nucleotide variant,NM_001205293.3(CACNA1E):c.3965C>T (p.Ser1322Phe),777,CACNA1E,HGNC:1392,Likely benign,0,-1,-,...,missense_variant,4856,3965,1322,S/F,tCc/tTc,COSV62407041,IMPACT=MODERATE;STRAND=1;SYMBOL=CACNA1E;SYMBOL...,181755373,NaN
2,2058909,single nucleotide variant,NM_002397.5(MEF2C):c.439A>G (p.Ile147Val),4208,MEF2C,HGNC:6996,Benign,0,-1,-,...,missense_variant,1090,439,147,I/V,Atc/Gtc,-,IMPACT=MODERATE;STRAND=-1;SYMBOL=MEF2C;SYMBOL_...,88752007,NaN
3,2058962,single nucleotide variant,NM_152296.5(ATP1A3):c.281T>C (p.Leu94Pro),478,ATP1A3,HGNC:801,Pathogenic,1,-1,-,...,missense_variant,419,281,94,L/P,cTg/cCg,-,IMPACT=MODERATE;STRAND=-1;SYMBOL=ATP1A3;SYMBOL...,41988012,NaN
4,2059150,single nucleotide variant,NM_001100.4(ACTA1):c.794A>G (p.Gln265Arg),58,ACTA1,HGNC:129,Pathogenic,1,-1,-,...,missense_variant,907,794,265,Q/R,cAg/cGg,CM992127,IMPACT=MODERATE;STRAND=-1;SYMBOL=ACTA1;SYMBOL_...,229432008,NaN


In [62]:
len(data_filter4_2[data_filter4_2.Consequence.isna()])

1619

In [63]:
data_filter4_2.Consequence.value_counts()

Consequence
missense_variant                                                 127328
missense_variant,splice_region_variant                             3742
missense_variant,NMD_transcript_variant                            1120
missense_variant,splice_region_variant,NMD_transcript_variant        46
Name: count, dtype: int64

In [64]:
# data_filter4 is the cleaned dataset before splitting by Assembly
# data_filter4_2 is the cleaned dataset after splitting, running VEP and merging again
len(data_filter4_2)==len(data_filter4)

True

In [99]:
data_filter4_2.Assembly.unique()

array(['GRCh38', 'GRCh37'], dtype=object)

In [66]:
data_filter4_2['gnomADe_AF'] = data_filter4_2['Extra'].str.extract(r'gnomADe_AF=([^;]+)')
data_filter4_2['gnomADg_AF'] = data_filter4_2['Extra'].str.extract(r'gnomADg_AF=([^;]+)')

data_filter4_2[['gnomADe_AF','gnomADg_AF']].head()

,gnomADe_AF,gnomADg_AF
0,0.001133,0.0009133
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


The columns 'gnomADe_AF' and 'gnomADg_AF' represent the AFs of the variant 
in the gnomAD Exomes (gnomADe_AF) and gnomAD Genomes (gnomADg_AF) datasets, respectively. 

The new column 'gnomAD_AF' combines these frequencies, prioritizing Exomes (gnomADe_AF) 
and using Genomes (gnomADg_AF) when Exomes data is missing. 

The final filtering step selects rare variants with a combined gnomAD_AF < 0.01.

With this we make sure we have a frequency value for every variant

In [67]:
data_filter4_2['gnomADe_AF'] = pd.to_numeric(data_filter4_2['gnomADe_AF'], errors='coerce')
data_filter4_2['gnomADg_AF'] = pd.to_numeric(data_filter4_2['gnomADg_AF'], errors='coerce')

# new column 'gnomAD_AF' that fills missing exomes AF with genomes AF
data_filter4_2['gnomAD_AF'] = data_filter4_2['gnomADe_AF'].fillna(data_filter4_2['gnomADg_AF'])

# filter for variants where gnomAD_AF < 0.01 and drop variants where gnomAD_AF is NaN
data_filter4_3 = data_filter4_2[data_filter4_2['gnomAD_AF'] < 0.01].dropna(subset=['gnomAD_AF'])

In [68]:
print(len(data_filter4_3))
data_filter4_3.head(2)

96686


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,RS# (dbSNP),nsv/esv (dbVar),...,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,Extra,PositionVCF_dashed,gnomADe_AF,gnomADg_AF,gnomAD_AF
0,1809239,single nucleotide variant,NM_001386125.1(OBSCN):c.496G>A (p.Ala166Thr),84033,OBSCN,HGNC:15719,Benign/Likely benign,0,-1,-,...,496,166,A/T,Gca/Aca,rs555146765,IMPACT=MODERATE;STRAND=1;SYMBOL=OBSCN;SYMBOL_S...,228212279,0.001133,0.000913,0.001133
6,2059238,single nucleotide variant,NM_006734.4(HIVEP2):c.1529C>T (p.Ser510Leu),3097,HIVEP2,HGNC:4921,Likely benign,0,-1,-,...,1529,510,S/L,tCa/tTa,rs531761193,IMPACT=MODERATE;STRAND=-1;SYMBOL=HIVEP2;SYMBOL...,142773210,0.000031,0.000013,0.000031


In [69]:
len(data_filter4_3[data_filter4_3.gnomAD_AF.notna()])

96686

In [70]:
print(data_filter4_3["Extra"].iloc[0])

IMPACT=MODERATE;STRAND=1;SYMBOL=OBSCN;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:15719;CCDS=CCDS1570.2;ENSP=ENSP00000284548;SWISSPROT=Q5VST9.202;UNIPARC=UPI0000425971;UNIPROT_ISOFORM=Q5VST9-3;HGVSc=ENST00000284548.16:c.496G>A;HGVSp=ENSP00000284548.11:p.Ala166Thr;AF=0.0004;AFR_AF=0.0008;AMR_AF=0.0014;EAS_AF=0;EUR_AF=0;SAS_AF=0;gnomADe_AF=0.001133;gnomADe_AFR_AF=0.0002016;gnomADe_AMR_AF=0.0004112;gnomADe_ASJ_AF=9.352e-05;gnomADe_EAS_AF=0;gnomADe_FIN_AF=0.0004933;gnomADe_MID_AF=0.0005405;gnomADe_NFE_AF=0.001353;gnomADe_REMAINING_AF=0.0005889;gnomADe_SAS_AF=0;gnomADg_AF=0.0009133;gnomADg_AFR_AF=0.0003389;gnomADg_AMI_AF=0;gnomADg_AMR_AF=0.001063;gnomADg_ASJ_AF=0;gnomADg_EAS_AF=0;gnomADg_FIN_AF=0.0001037;gnomADg_MID_AF=0;gnomADg_NFE_AF=0.00156;gnomADg_REMAINING_AF=0.0004812;gnomADg_SAS_AF=0;MAX_AF=0.00156;MAX_AF_POPS=gnomADg_NFE;CLIN_SIG=likely_benign,benign;PHENO=1


In [71]:
data_filter4_3 = data_filter4_3.sort_values(by='LastEvaluated (Year)', ascending=False)
data_filter4_3['HGVSp'] = data_filter4_3['Extra'].str.extract(r'HGVSp=([^;]+)')

cols_check = ['Type', 'GeneID','Gene', 'GeneSymbol', 'Feature','HGNC_ID',
      'Assembly','ChromosomeAccession', 'Chromosome', 
      'HGVSp','Protein_position', 'Amino_acids','Variant']

data_filter4_4 = data_filter4_3.drop_duplicates(subset=cols_check, keep='first')

print(len(data_filter4_3))
print(len(data_filter4_4))

96686
96544


### 5. Keep genes with at least one pathogenic variant of any type.

In [72]:
data_filter4_3.ClinicalSignificance.unique()

array(['Pathogenic/Likely pathogenic', 'Likely pathogenic', 'Pathogenic',
       'Likely benign', 'Benign', 'Benign/Likely benign'], dtype=object)

In [73]:
def filter_pathogenic_genes(df):
    # pathogenic terms
    pathogenic_terms = ['Pathogenic', 'Likely pathogenic', 'Pathogenic/Likely pathogenic']
    
    # filter for genes that have at least one pathogenic variant
    pathogenic_genes = df[df['ClinicalSignificance'].isin(pathogenic_terms)]['GeneSymbol'].unique()
    
    # filter df to retain only variants for those genes
    df_filtered = df[df['GeneSymbol'].isin(pathogenic_genes)]
    
    return df_filtered

In [74]:
data_filter5 = filter_pathogenic_genes(data_filter4_4)

In [75]:
len(data_filter5)

49187

Notice that only GRCh38 remains. All the pathogenic variants in the filtered dataset are from this assembly.

In [106]:
data_filter5["Assembly"].value_counts()

Assembly
GRCh38    49187
Name: count, dtype: int64

### Some parsing final steps...

In [76]:
def classify_significance(clinical_significance):
    if any(term in clinical_significance for term in ['Pathogenic', 'Likely pathogenic']):
        return 'P'
    elif any(term in clinical_significance for term in ['Benign', 'Likely benign']):
        return 'B'
    return 'Other'

In [77]:
cleaned_ClinVar_dataset = data_filter5.copy()
cleaned_ClinVar_dataset['BinaryClinicalSignificance'] = data_filter5['ClinicalSignificance'].apply(classify_significance)

In [78]:
cleaned_ClinVar_dataset.BinaryClinicalSignificance.value_counts()

BinaryClinicalSignificance
B    34405
P    14782
Name: count, dtype: int64

In [79]:
len(cleaned_ClinVar_dataset.GeneSymbol.unique())

2156

In [93]:
len(cleaned_ClinVar_dataset)

49187

In [81]:
cleaned_ClinVar_dataset.head()

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,RS# (dbSNP),nsv/esv (dbVar),...,Amino_acids,Codons,Existing_variation,Extra,PositionVCF_dashed,gnomADe_AF,gnomADg_AF,gnomAD_AF,HGVSp,BinaryClinicalSignificance
119639,29901,single nucleotide variant,NM_182894.3(VSX2):c.679C>T (p.Arg227Trp),338917,VSX2,HGNC:1975,Pathogenic/Likely pathogenic,1,121912545,-,...,R/W,Cgg/Tgg,"rs121912545,CM042327",IMPACT=MODERATE;STRAND=1;SYMBOL=VSX2;SYMBOL_SO...,74259701,0.000003,0.000020,0.000003,ENSP00000261980.2:p.Arg227Trp,P
93641,104480,single nucleotide variant,NM_000180.4(GUCY2D):c.307G>A (p.Glu103Lys),3000,GUCY2D,HGNC:4689,Likely pathogenic,1,61749668,-,...,E/K,Gag/Aag,"rs61749668,CM077936",IMPACT=MODERATE;STRAND=1;SYMBOL=GUCY2D;SYMBOL_...,8003354,0.000191,0.000033,0.000191,ENSP00000254854.4:p.Glu103Lys,P
59576,3734350,single nucleotide variant,NM_000441.2(SLC26A4):c.1335G>C (p.Leu445Phe),5172,SLC26A4,HGNC:8818,Likely pathogenic,1,-1,-,...,L/F,ttG/ttC,rs1355468475,IMPACT=MODERATE;STRAND=1;SYMBOL=SLC26A4;SYMBOL...,107694474,NaN,0.000007,0.000007,ENSP00000494017.1:p.Leu445Phe,P
59575,3734348,single nucleotide variant,NM_000441.2(SLC26A4):c.1279T>C (p.Ser427Pro),5172,SLC26A4,HGNC:8818,Pathogenic,1,-1,-,...,S/P,Tct/Cct,rs758015694,IMPACT=MODERATE;STRAND=1;SYMBOL=SLC26A4;SYMBOL...,107694418,0.000002,NaN,0.000002,ENSP00000494017.1:p.Ser427Pro,P
59572,3734343,single nucleotide variant,NM_000441.2(SLC26A4):c.1207G>T (p.Ala403Ser),5172,SLC26A4,HGNC:8818,Likely pathogenic,1,-1,-,...,A/S,Gcc/Tcc,"rs1791527351,COSV107219136",IMPACT=MODERATE;STRAND=1;SYMBOL=SLC26A4;SYMBOL...,107690181,0.000001,NaN,0.000001,ENSP00000494017.1:p.Ala403Ser,P


### And some statistics

In [82]:
def count_p_and_b_per_gene(df):
    # group by 'GeneSymbol' and 'BinaryClinicalSignificance', then count
    counts = df.groupby(['GeneSymbol', 'BinaryClinicalSignificance']).size().unstack(fill_value=0)

    if 'B' in counts.columns and 'P' in counts.columns:
        counts.columns = ['B_count', 'P_count']
    elif 'P' in counts.columns:
        counts = counts.rename(columns={'P': 'P_count'})
    elif 'B' in counts.columns:
        counts = counts.rename(columns={'B': 'B_count'})
    
    return counts

In [83]:
counts_per_gene = count_p_and_b_per_gene(cleaned_ClinVar_dataset)
counts_per_gene

,B_count,P_count
GeneSymbol,,
AAAS,2,8
AARS1,21,4
AARS2,19,6
AASS,9,1
ABAT,2,3
...,...,...
ZMYND10,9,4
ZMYND11,12,2
ZNF341,21,1


In [84]:
counts_per_gene[(counts_per_gene.P_count>=30) & (counts_per_gene.B_count>= 30)].sort_values(by=['P_count','B_count'], ascending=False)

,B_count,P_count
GeneSymbol,,
USH2A,90,133
TP53,65,86
COL4A3,36,83
PKHD1,38,79
COL7A1,237,76
FBN1,48,68
COL4A4,47,68
RYR1,47,62
COL4A5,113,60


In [85]:
cleaned_ClinVar_dataset['PositionVCF'] = pd.to_numeric(cleaned_ClinVar_dataset['PositionVCF'])

# sort by chromosome and position
cleaned_ClinVar_dataset = cleaned_ClinVar_dataset.sort_values(['Chromosome', 'PositionVCF'])
cleaned_ClinVar_dataset = cleaned_ClinVar_dataset.reset_index(drop=True)

In [96]:
cleaned_ClinVar_dataset["Assembly"].unique()

array(['GRCh38'], dtype=object)

In [86]:
# this works as database (with the applied filters) from where to retrieve variants for the variants_pipeline.sh
cleaned_ClinVar_dataset.to_csv('../data/clinvar/cleaned_ClinVar_dataset.csv', index=0)

In [87]:
cleaned_ClinVar_dataset[['Name','Chromosome','ReferenceAlleleVCF','PositionVCF','AlternateAlleleVCF']].head()

,Name,Chromosome,ReferenceAlleleVCF,PositionVCF,AlternateAlleleVCF
0,NM_198576.4(AGRN):c.11G>C (p.Arg4Pro),1,G,1020183,C
1,NM_198576.4(AGRN):c.125A>C (p.Glu42Ala),1,A,1020297,C
2,NM_198576.4(AGRN):c.494C>T (p.Pro165Leu),1,C,1035307,T
3,NM_198576.4(AGRN):c.773C>T (p.Thr258Ile),1,C,1041218,T
4,NM_198576.4(AGRN):c.1058A>G (p.Gln353Arg),1,A,1041583,G


Now that the dataset is cleaned, we convert it into VCF format for input to the VEP tool, so we can obtain predictions from the chosen pathogenicity predictors.

In [88]:
create_vcf(cleaned_ClinVar_dataset[['Chromosome', 'PositionVCF', 'RS# (dbSNP)', 
                                    'ReferenceAlleleVCF', 'AlternateAlleleVCF']],
                                    "../data/clinvar/cleaned_Clinvar_dataset_inputVEP.vcf")

VCF file created at: ../data/clinvar/cleaned_Clinvar_dataset_inputVEP.vcf


Command run in the script (notice separation by assembly)

In [90]:
#    ./vep -i cleaned_Clinvar_dataset_inputVEP.vcf -o cleaned_Clinvar_dataset_outputVEP.txt --offline \
#        --assembly $assembly \
#        --symbol --transcript_version --ccds --protein --uniprot --canonical \
#        --hgvs --fasta Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz \
#        --af --af_1kg --af_gnomade --af_gnomadg --max_af \
#        --sift b --polyphen b \
#        --plugin AlphaMissense,file=/home/aitanadiaz/ensembl-vep/plugins/AlphaMissense_${assembly}.tsv.gz \
#        --plugin BayesDel,file=/home/aitanadiaz/ensembl-vep/plugins/BayesDel_170824_addAF_all_scores.txt.gz \
#        --plugin Blosum62 \
#        --plugin CADD,snv=/home/aitanadiaz/ensembl-vep/plugins/whole_genome_SNVs_${assembly}.tsv.gz \
#        --plugin dbNSFP,/home/aitanadiaz/ensembl-vep/plugins/dbNSFP5.0a.gz,VEST4_score,VEST4_rankscore \
#        --plugin EVE,file=/home/aitanadiaz/ensembl-vep/plugins/EVE/eve_merged.vcf.gz \
#        --plugin REVEL,file=/home/aitanadiaz/ensembl-vep/plugins/new_tabbed_revel_${assembly}.tsv.gz

### Read VEP output (predictions added!)

Notice we are obtaining predictions only for the following predictors:
- __CADD__
- __EVE__
- __AlphaMissense__
- __BayesDel__
- __REVEL__
- __VEST4__

Missing ones will be run and add after parsing VEP results.

In [ ]:
VEP_output = pd.read_csv('../data/clinvar/TTiZfTr7N4guoG4E.txt', sep='\t')

In [ ]:
VEP_output.columns()

In [ ]:
# List of target predictors to check for in column names
predictors = [
    'SIFT', 'PolyPhen', 'MutPred', 'MutationAssessor', 'CADD', 'MetaSVM', 'MetaLR',
    'REVEL', 'Envision', 'FATHMM', 'EVE', 'VARITY', 'VEST4', 'PrimateAI', 'BayesDel', 
    'VariPred', '3Cnet', 'CAPICE', 'SNPred', 'AlphaMissense', 'MutScore'
]

# Loop through the predictors and check if any of them exist in the column names
for predictor in predictors:
    # Check if the predictor name is part of any column name
    matching_columns = [col for col in VEP_output.columns if predictor in col]
    
    # If any matching columns are found, print their head
    if matching_columns:
        print(f"Columns matching '{predictor}':")
        print(VEP_output[matching_columns].head())
        print("\n" + "="*50 + "\n")

In [ ]:
df = VEP.clean_vep_data(VEP_output)

In [ ]:
df.head()
print(len(df))

In [ ]:
def check_coverage(df):
    # check coverage
    cols=[i for i in df.columns if '_binary' in i]
    
    adding=[]
    for i in cols:  
        tmp = df[df[i].notna()]
        i =i.split('_')[0]
        if i == 'am':
            i = 'AlphaMissense'  
        coverage = round(100 * len(tmp) / len(df), 2) 
        adding.append([i, coverage])
        
    table=pd.DataFrame(adding, columns=['Predictor','Coverage'])
    table=table.sort_values('Coverage',ascending=False).reset_index(drop=True)
    return table

In [ ]:
clinvar = pd.read_csv('../data/clinvar/cleaned_ClinVar_dataset.csv')
len(clinvar)

In [ ]:
for col in clinvar.columns:
    if col in df.columns:
        print(col)

In [ ]:
def adjustments_before_merging(clinvar,df):
    # adjustments in the clinvar dataset
    clinvar_to_merge = clinvar.copy()
    clinvar_to_merge['Protein_position'] = clinvar_to_merge['Protein_position'].astype(int)
    
    # adjustments in the VEP output
    df_to_merge=df.copy()
    df_to_merge['Chromosome'] = df_to_merge.apply(lambda x: str(x['Location'].split(':')[0]), axis=1)
    df_to_merge['Protein_position'] = df_to_merge['Protein_position'].astype(int)
    df_to_merge.rename(columns={'SYMBOL':'GeneSymbol'}, inplace=True)
    df_to_merge2=df_to_merge.drop_duplicates(subset=['#Uploaded_variation', 'Location', 'Allele', 'GeneSymbol', 'Gene','Feature_type',
                  'Protein_position', 'Amino_acids','Chromosome'], keep='first').reset_index(drop=True)
    
    print(len(df_to_merge),len(df_to_merge2))
    
    # '-' in clinvar_to_merge['MANE_SELECT'].unique()
    
    #del df_to_merge['Consequence']
    #del df_to_merge['MANE_SELECT']
    
    del df_to_merge2['Consequence']
    del df_to_merge2['MANE_SELECT']
    
    return clinvar_to_merge, df_to_merge2

In [ ]:
clinvar_merge, df_merge = adjustments_before_merging(clinvar, df)

In [ ]:
df_merge.BIOTYPE.unique()

In [ ]:
merge_cols=['HGNC_ID','Gene','GeneSymbol','Chromosome','Protein_position','Amino_acids','variant','Feature']
#merge_cols=['HGNC_ID','Gene','Feature','GeneSymbol','Chromosome','Protein_position','Amino_acids']

print(clinvar_merge[merge_cols].dtypes)
print(len(clinvar_merge),'\n')

print(df_merge[merge_cols].dtypes)
print(len(df_merge))

In [ ]:
clinvar_merge['Chromosome'] = clinvar_merge['Chromosome'].astype(str)
df_merge['Chromosome'] = df_merge['Chromosome'].astype(str)

print(clinvar_merge[merge_cols].dtypes)
print(df_merge[merge_cols].dtypes)

In [ ]:
merged_inner = clinvar_merge.merge(df_merge, on=merge_cols, how='inner')
print(len(merged_inner))

check_coverage(merged_inner)

In [ ]:
merged_inner.BinaryClinicalSignificance.value_counts()

In [ ]:
merged_inner[['GeneSymbol','Feature','Uniprot_acc', 'Uniprot_entry','UNIPROT_ISOFORM']]

In [ ]:
def process_and_harmonize_data(df):
    """
    Final steps to process protein data:
    1. Identify canonical entries
    2. Filter for canonical proteins
    3. Filter for majority features
    4. Harmonize remaining UniProt discrepancies
    
    Parameters:
    df: DataFrame with required columns: 
        UNIPROT_ISOFORM, Uniprot_acc, GeneSymbol, Features
    
    Returns:
    DataFrame: processed and harmonized DataFrame
    """
    def get_canonical_info(row):
        isoform = str(row['UNIPROT_ISOFORM'])

        if isoform.endswith('-1'):
            return 'canonical'
        elif isoform == '-':
            acc_list = str(row['Uniprot_acc']).split(',')
            canonical = [acc for acc in acc_list if len(acc) == 6 and acc[0] in ['P', 'Q', 'O']]
            return 'canonical' if canonical else 'not_canonical'
        else:
            return 'not_canonical'
    
    def get_uniprot_prioritizing_isoform(row):
        isoform = str(row['UNIPROT_ISOFORM'])

        if isoform != '-':
            if '-' in isoform:
                return isoform.split('-')[0]
            return isoform
        
        acc_list = str(row['Uniprot_acc']).split(',')
        canonical = [acc for acc in acc_list if len(acc) == 6 and acc[0] in ['P', 'Q', 'O']]
        if canonical:
            return canonical[0]
        
        # if no canonical found, return first entry w/o isoform number
        first_acc = acc_list[0]
        if '-' in first_acc:
            return first_acc.split('-')[0]
        return first_acc
    
    def filter_to_majority_features(df):
        majority_features = df.groupby('GeneSymbol').agg({
            'Feature': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None
        }).reset_index()

        gene_to_majority_feature = dict(zip(majority_features['GeneSymbol'], 
                                          majority_features['Feature']))

        mask = df.apply(lambda row: (row['Feature'] == gene_to_majority_feature[row['GeneSymbol']]), 
                       axis=1)
        return df[mask].copy()
    
    def harmonize_uniprot_mappings(df):
        gene_uniprot_counts = df.groupby('GeneSymbol')['uniprot'].agg(['nunique', 'unique'])
        genes_to_fix = gene_uniprot_counts[gene_uniprot_counts['nunique'] > 1].index
        
        if len(genes_to_fix) == 0:
            return df

        fixed_df = df.copy()
        for gene in genes_to_fix:
            gene_data = fixed_df[fixed_df['GeneSymbol'] == gene]
            majority_uniprot = gene_data['uniprot'].mode().iloc[0]
            fixed_df.loc[fixed_df['GeneSymbol'] == gene, 'uniprot'] = majority_uniprot
            
        return fixed_df
    
    # initial stats
    print("Initial statistics:")
    print(f"Total rows: {len(df)}")
    print(f"Unique genes: {df['GeneSymbol'].nunique()}")
    print(f"Unique UniProt IDs: {df['Uniprot_acc'].nunique()}")
    
    # canonical information
    df['canonical_status'] = df.apply(get_canonical_info, axis=1)
    
    # filter for canonical entries
    canonical_df = df[df['canonical_status'] == 'canonical'].copy()
    print("\nAfter canonical filtering:")
    print(f"Remaining rows: {len(canonical_df)}")
    
    # standardized UniProt IDs
    canonical_df['uniprot'] = canonical_df.apply(get_uniprot_prioritizing_isoform, axis=1)
    
    # filter for majority features
    feature_filtered_df = filter_to_majority_features(canonical_df)
    print("\nAfter feature filtering:")
    print(f"Remaining rows: {len(feature_filtered_df)}")
    
    # check and fix any remaining UniProt discrepancies
    final_df = harmonize_uniprot_mappings(feature_filtered_df)
    
    # final validation
    print("\nFinal statistics:")
    print(f"Final rows: {len(final_df)}")
    print(f"Unique genes: {final_df['GeneSymbol'].nunique()}")
    print(f"Unique UniProt IDs: {final_df['uniprot'].nunique()}")
    
    if final_df['GeneSymbol'].nunique() != final_df['uniprot'].nunique():
        print("\nThere are still differences in unique gene and UniProt counts!")
    
    return final_df

In [ ]:
clin_final = process_and_harmonize_data(merged_inner)

In [ ]:
print(clin_final.BinaryClinicalSignificance.value_counts())
print(len(clin_final))
print(clin_final.uniprot.nunique())
print(clin_final['GeneSymbol'].nunique())

In [ ]:
check_coverage(clin_final)

In [ ]:
clin_final.to_csv('../data/clinvar/cleaned_Clinvar_dataset_VEPoutput.csv',index=0)